# Grab's Case Study
<h2> Economics, Mobility & Deliveries </h2>
Take-home Analysis Task


<h3>Instructions:<br></h3>
You have three hours from receiving this assignment to complete the task below and return
your completed work. At the end of three hours, please submit your analysis output including
any notebooks, slides, visuals, and code back to youzhen.chong@grab.com and
zheching.tan@grab.com.<br>

Given the dataset of bookings, your task is to:
1. Explore and understand the dataset.
2. Prepare a model or analysis that explains the number of bids a given booking received.
Be prepared to explain and defend your model choices.
3. Based on your results in 2), suggest possible implications of your analysis and prepare a
business strategy or recommendation that would make sense. Please present your
findings concisely in slide format. Assume that your audience comprises leaders of the
Singapore Grab business. Please limit to a maximum of 5 slides, but you do not need to
use all 5.
4. Suggest any other data, internal or external, that may help your conclusion.
5. Detail any additional insights or recommendations that may come up in your analysis of
the data.


<h3>Dataset description:</h3> <br>
The attached dataset contains a simplified subset of the bookings attempted in Singapore
between 7am and 8am in Feb 2015. Each row indicates a booking attempt made by a
passenger. A job is broadcast to potentially available drivers in a nearby area. Drivers can elect
to bid to be allocated that job. We are interested in any relationships that explain the number of
bids from drivers that a job received. The data is in CSV format.<br>
<h4>Variables:</h4>

- distance trip distance
- pick_up_time_local time of passenger pickup
- bids number of bids made for each attempted booking by available drivers. If bids are
0 it implies that no drivers made a bid for that job.
- PICKUP_DIS code for district in which trip would start
- pick_up_district_name name of district in which trip would start
- DROP_OFF_DIS code for district in which trip would end
- drop_off_district_name name of district in which trip would end
- hours hour interval of day
- pick_up_lat_round latitude of pickup location, rounded to two decimal places or
approximately 1km
- pick_up_long_round longitude of pickup location, rounded to two decimal places or
approximately 1km
- drop_off_lat_round latitude of dropoff location, rounded to two decimal places or
approximately 1km
- drop_off_long_round longitude of dropoff location, rounded to two decimal places
or approximately 1km<br><br>

Note: you may not need to use all variables to answer the question sufficiently.
Map of district locations by district code:
https://data.gov.sg/dataset/sla-land-survey-district<br>
Notes and hints:
- The contents of the test as well as the dataset are confidential.
- This is a 3-hour test but you shouldn’t need the entire time to do the analysis. Make sure
to leave enough time to think about how to deliver your findings effectively.
- You may use any language, package, framework, or tools for your analysis but
remember that your code has to be readable, understandable, and reproducible. It is
always in your benefit to explain what you are doing and why.
- You may use any publicly available external datasets to complement your analysis but it
is not necessary to answer the primary problem.
-  You may search the internet to help you with your work.

# Initialization

In [1]:
import pandas as pd
import numpy as np

In [35]:
input_df = pd.read_csv('grab_dataset.csv').drop(columns={'Unnamed: 0'})
input_df.columns=input_df.columns.str.lower()
print(input_df.shape)
input_df.head()

(50751, 12)


,distance,pick_up_time_local,bids,pickup_dis,pick_up_district_name,drop_off_dis,drop_off_district_name,hours,pick_up_lat_round,pick_up_long_round,drop_off_lat_round,drop_off_long_round
0,17.687,9/2/2015 07:01:25,2,MK20,Seletar,MK02,Farrer Road,7,1.39,103.84,1.30,103.82
1,12.747,12/2/2015 07:19:26,0,MK05,Jurong East,MK02,Farrer Road,7,1.32,103.74,1.30,103.81
2,20.560,23/2/2015 07:16:59,0,MK21,Punggol/Sengkang,MK02,Farrer Road,7,1.39,103.90,1.30,103.81
3,2.967,18/2/2015 07:59:36,9,TS21,City,MK02,Farrer Road,7,1.30,103.84,1.31,103.82
4,22.701,2/2/2015 07:30:00,1,MK13,Woodlands,MK02,Farrer Road,7,1.44,103.79,1.30,103.81


In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50751 entries, 0 to 50750
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   distance                50751 non-null  float64
 1   pick_up_time_local      50751 non-null  object 
 2   bids                    50751 non-null  int64  
 3   pickup_dis              50751 non-null  object 
 4   pick_up_district_name   50751 non-null  object 
 5   drop_off_dis            50741 non-null  object 
 6   drop_off_district_name  50741 non-null  object 
 7   hours                   50751 non-null  int64  
 8   pick_up_lat_round       50751 non-null  float64
 9   pick_up_long_round      50751 non-null  float64
 10  drop_off_lat_round      50751 non-null  float64
 11  drop_off_long_round     50751 non-null  float64
 12  bookings                50751 non-null  int64  
 13  unallocated             50751 non-null  int64  
dtypes: float64(5), int64(4), object(5)
mem

# Exploratory Data Analysis

In [100]:
df = input_df.copy().rename(columns={'pick_up_district_name':'pickup', 'drop_off_district_name':'dropoff'})
df['bookings'] = 1
df['unallocated'] = np.where(df.bids==0, 1, 0)
df['allocated'] = np.where(df.bids>0, 1,0)
df['distance_ceiled'] = (np.ceil(df.distance)).astype(int)
# Pickup dropoff
df['pudo'] = (df.pickup + '-'+ df.dropoff).str.lower().str.replace(" ", "_")

df.head()

,distance,pick_up_time_local,bids,pickup_dis,pickup,drop_off_dis,dropoff,hours,pick_up_lat_round,pick_up_long_round,drop_off_lat_round,drop_off_long_round,bookings,unallocated,allocated,distance_ceiled,pudo
0,17.687,9/2/2015 07:01:25,2,MK20,Seletar,MK02,Farrer Road,7,1.39,103.84,1.30,103.82,1,0,1,18,seletar-farrer_road
1,12.747,12/2/2015 07:19:26,0,MK05,Jurong East,MK02,Farrer Road,7,1.32,103.74,1.30,103.81,1,1,0,13,jurong_east-farrer_road
2,20.560,23/2/2015 07:16:59,0,MK21,Punggol/Sengkang,MK02,Farrer Road,7,1.39,103.90,1.30,103.81,1,1,0,21,punggol/sengkang-farrer_road
3,2.967,18/2/2015 07:59:36,9,TS21,City,MK02,Farrer Road,7,1.30,103.84,1.31,103.82,1,0,1,3,city-farrer_road
4,22.701,2/2/2015 07:30:00,1,MK13,Woodlands,MK02,Farrer Road,7,1.44,103.79,1.30,103.81,1,0,1,23,woodlands-farrer_road


In [203]:
allocation_rate = df.allocated.sum()/df.bookings.sum()
unallocation_rate = 1-allocation_rate
print('Allocation Rate:', allocation_rate)
print('Unallocation Rate:', unallocation_rate)

Allocation Rate: 0.7282812161336722
Unallocation Rate: 0.27171878386632775


In [142]:
def aggregator(df, groupBy, type_):
    
    if type_ == 'area':
        aggregation = {'distance':'median'
                                  , 'bookings':'sum'
                                  , 'bids':'sum'
                                  , 'allocated':'sum'                            
                                  , 'unallocated':'sum'}
    else:  
         aggregation = {'bookings':'sum'
                                  , 'bids':'sum'
                                  , 'allocated':'sum'                            
                                  , 'unallocated':'sum'}
        
    temp_df = df.groupby([groupBy]
                           ).agg(aggregation).sort_values('bids', ascending=False
                                               ).reset_index()#.rename(columns={'pick_up_district_name':'pick_up'})

    temp_df['allocation_rate'] = temp_df.allocated/temp_df.bookings
    temp_df['unallocation_rate'] = 1-temp_df.allocation_rate
    temp_df['weighted_unallocation_rate'] = temp_df.unallocation_rate * (temp_df.bookings/temp_df.bookings.sum())
    return temp_df

pudo_agg = aggregator(df, 'pudo', 'area')
distance_agg = aggregator(df, 'distance_ceiled', 'distance').sort_values('distance_ceiled').reset_index(drop=True)
distance_agg = distance_agg[(np.abs(stats.zscore(distance_agg)) < 3).all(axis=1)]
pickup_agg.head()

,pickup,distance,bookings,bids,allocated,unallocated,allocation_rate,unallocation_rate,weighted_unallocation_rate
0,City,5.5255,8780,13803,6476,2304,0.737585,0.262415,0.045398
1,Punggol/Sengkang,14.2195,4018,8255,3344,674,0.832255,0.167745,0.013281
2,Woodlands,18.9260,3667,7029,2948,719,0.803927,0.196073,0.014167
3,Ang Mo Kio/Bishan,12.4260,2955,4061,2145,810,0.725888,0.274112,0.015960
4,Jurong West,11.4000,2245,4060,1781,464,0.793318,0.206682,0.009143


In [304]:

pickup_agg = aggregator(df, 'pickup', 'area')
pickup_agg['area_type'] = 'pickup'
pickup_agg['is_high_booking'] = np.where(pickup_agg.bookings>=pickup_agg.bookings.median(),1,0)
pickup_agg['is_good_allocation'] = np.where(pickup_agg.unallocation_rate<pickup_agg.weighted_unallocation_rate.sum(),1,0)
pickup_agg['quadrant'] = np.where((pickup_agg.is_high_booking==1) & (pickup_agg.is_good_allocation==1), 'Role Models'
                                  , np.where((pickup_agg.is_high_booking==1) & (pickup_agg.is_good_allocation==0), 'Dying'
                                            , np.where((pickup_agg.is_high_booking==0) & (pickup_agg.is_good_allocation==1), 'Self-Sustain', 'Optimize Last')
                                            )
                                 )
pickup_agg

,pickup,distance,bookings,bids,allocated,unallocated,allocation_rate,unallocation_rate,weighted_unallocation_rate,area_type,is_high_booking,is_good_allocation,quadrant
0,City,5.5255,8780,13803,6476,2304,0.737585,0.262415,0.045398,pickup,1,1,Role Models
1,Punggol/Sengkang,14.2195,4018,8255,3344,674,0.832255,0.167745,0.013281,pickup,1,1,Role Models
2,Woodlands,18.9260,3667,7029,2948,719,0.803927,0.196073,0.014167,pickup,1,1,Role Models
3,Ang Mo Kio/Bishan,12.4260,2955,4061,2145,810,0.725888,0.274112,0.015960,pickup,1,0,Dying
4,Jurong West,11.4000,2245,4060,1781,464,0.793318,0.206682,0.009143,pickup,1,1,Role Models
5,Yishun,18.2945,1686,3856,1413,273,0.838078,0.161922,0.005379,pickup,1,1,Role Models
6,Fort Road,8.9550,2278,3691,1843,435,0.809043,0.190957,0.008571,pickup,1,1,Role Models
7,Chua Chu Kang,16.4785,2298,3650,1675,623,0.728895,0.271105,0.012276,pickup,1,1,Role Models
8,Bedok,12.7440,1281,3439,1179,102,0.920375,0.079625,0.002010,pickup,1,1,Role Models
9,Hougang,12.5980,1799,2710,1336,463,0.742635,0.257365,0.009123,pickup,1,1,Role Models


In [303]:

dropoff_agg = aggregator(df, 'dropoff', 'area')
dropoff_agg['area_type'] = 'dropoff'

dropoff_agg['is_high_booking'] = np.where(dropoff_agg.bookings>=dropoff_agg.bookings.median(),1,0)
dropoff_agg['is_good_allocation'] = np.where(dropoff_agg.unallocation_rate<dropoff_agg.weighted_unallocation_rate.sum(),1,0)
dropoff_agg['quadrant'] = np.where((dropoff_agg.is_high_booking==1) & (dropoff_agg.is_good_allocation==1), 'Role Models'
                                  , np.where((dropoff_agg.is_high_booking==1) & (dropoff_agg.is_good_allocation==0), 'Dying'
                                            , np.where((dropoff_agg.is_high_booking==0) & (dropoff_agg.is_good_allocation==1), 'Self-Sustain', 'Optimize Last')
                                            )
                                 )
dropoff_agg

,dropoff,distance,bookings,bids,allocated,unallocated,allocation_rate,unallocation_rate,weighted_unallocation_rate,area_type,is_high_booking,is_good_allocation,quadrant
0,City,9.5820,17216,22228,11435,5781,0.664208,0.335792,0.113932,dropoff,1,0,Dying
1,Loyang Road,21.0530,4001,9143,3394,607,0.848288,0.151712,0.011963,dropoff,1,1,Role Models
2,Woodlands,9.5280,2202,6211,2041,161,0.926885,0.073115,0.003173,dropoff,1,1,Role Models
3,Ang Mo Kio/Bishan,9.6970,2242,4195,1846,396,0.823372,0.176628,0.007804,dropoff,1,1,Role Models
4,Queenstown,11.3200,3286,3888,2080,1206,0.632988,0.367012,0.023768,dropoff,1,0,Dying
5,Jurong East,11.4870,2134,2949,1528,606,0.716026,0.283974,0.011943,dropoff,1,0,Dying
6,Tiong Bahru,14.8370,2052,2850,1441,611,0.702242,0.297758,0.012042,dropoff,1,0,Dying
7,Yishun,8.9790,1003,2733,928,75,0.925224,0.074776,0.001478,dropoff,1,1,Role Models
8,Upper East Coast Road,14.2390,1477,2689,1242,235,0.840894,0.159106,0.004631,dropoff,1,1,Role Models
9,Toa Payoh,10.2890,1436,1955,1006,430,0.700557,0.299443,0.008474,dropoff,1,0,Dying


In [319]:
area_agg= pd.concat([pickup_agg.rename(columns={'pickup':'area'})
                   , dropoff_agg.rename(columns={'dropoff':'area'})])
area_agg


,area,distance,bookings,bids,allocated,unallocated,allocation_rate,unallocation_rate,weighted_unallocation_rate,area_type,is_high_booking,is_good_allocation,quadrant
0,City,5.5255,8780,13803,6476,2304,0.737585,0.262415,0.045398,pickup,1,1,Role Models
1,Punggol/Sengkang,14.2195,4018,8255,3344,674,0.832255,0.167745,0.013281,pickup,1,1,Role Models
2,Woodlands,18.9260,3667,7029,2948,719,0.803927,0.196073,0.014167,pickup,1,1,Role Models
3,Ang Mo Kio/Bishan,12.4260,2955,4061,2145,810,0.725888,0.274112,0.015960,pickup,1,0,Dying
4,Jurong West,11.4000,2245,4060,1781,464,0.793318,0.206682,0.009143,pickup,1,1,Role Models
...,...,...,...,...,...,...,...,...,...,...,...,...,...
28,Farrer Road,11.2860,315,326,186,129,0.590476,0.409524,0.002542,dropoff,0,0,Optimize Last
29,Diary Farm,8.0790,147,240,109,38,0.741497,0.258503,0.000749,dropoff,0,1,Self-Sustain
30,Lim Chu Kang,14.5380,107,139,76,31,0.710280,0.289720,0.000611,dropoff,0,0,Optimize Last
31,Tampines Road,13.2230,10,18,8,2,0.800000,0.200000,0.000039,dropoff,0,1,Self-Sustain


In [381]:
improvement_target = 0.1
segment_target = 'Role Models'

area_summary = area_agg.groupby(['area_type','quadrant']).agg({'unallocation_rate':'median'
                                                              , 'bookings':"median"}).reset_index()

area_summary['simulated_unallocation_rate'] = np.where(area_summary.quadrant== segment_target
                                                       , area_summary.unallocation_rate - improvement_target
                                                      , area_summary.unallocation_rate)

current_unallocation_rate =(area_summary.unallocation_rate * (area_summary.bookings/area_summary.bookings.sum())).sum()
simulated_unallocation_rate = (area_summary.simulated_unallocation_rate * (area_summary.bookings/area_summary.bookings.sum())).sum()


impact = simulated_unallocation_rate/current_unallocation_rate-1
text = 'Decrease of Unallocation Rate by '+ str(improvement_target) + ' will have impact of overall Unallocation rate by ' + str(abs(round(impact,3)))
# print(simulated_unallocation_rate/current_unallocation_rate-1)

print(text)

Decrease of Unallocation Rate by 0.1 will have impact of overall Unallocation rate by 0.126


In [380]:
area_agg.quadrant.unique()

array(['Role Models', 'Dying', 'Self-Sustain', 'Optimize Last'],
      dtype=object)

-0.08234969365099032

In [343]:
import plotly.express

fig = px.bar(area_summary, x ='quadrant', y='unallocation_rate', color = 'area_type',  barmode='group',)
fig.update_layout(title='Unallocation Rate by Quadrant', title_x = 0.5, width = 600, height =400
                    , legend = dict(orientation = "h", yanchor = 'bottom', xanchor = 'center' , x = 0.5 , y =1, title='')
)
fig.update_xaxes(categoryorder='array'
                 , categoryarray= ['Role Models', 'Dying', 'Optimize Last', 'Self-Sustain'])


fig.show()

In [344]:
import plotly.express

fig = px.bar(area_summary, x ='quadrant', y='bookings', color = 'area_type',  barmode='group',)
fig.update_layout(title='Bookings  by Quadrant', title_x = 0.5, width = 600, height =400
                    , legend = dict(orientation = "h", yanchor = 'bottom', xanchor = 'center' , x = 0.5 , y =1, title='')
)

fig.update_xaxes(categoryorder='array'
                 , categoryarray= ['Role Models', 'Dying', 'Optimize Last', 'Self-Sustain'])


fig.show()

In [123]:


from scipy import stats
from scipy.stats import zscore


In [191]:
pickup_agg.bookings.median()

1096.0

In [205]:
pickup_agg.weighted_unallocation_rate.sum()

0.27171878386632775

In [258]:
import plotly.express as px
fig = px.scatter(pickup_agg, y='unallocation_rate'
                 , x='bookings'
                 , hover_data=['pickup']
                 , log_x=True
)
fig.update_traces(textposition='top center')

fig.add_vrect(x0=pickup_agg.bookings.median()
              , x1=pickup_agg.bookings.median()
              , line_dash="dot",)

temp_txt= "Unallocation Rate: " + str(round(pickup_agg.weighted_unallocation_rate.sum(),2))
fig.add_hrect(y0=pickup_agg.weighted_unallocation_rate.sum()
              , y1=pickup_agg.weighted_unallocation_rate.sum()
              , annotation_text=temp_txt
              , annotation_position="top left"
              , line_dash="dot")

fig.update_layout(title='Unallocation Rate by PickupArea', title_x = 0.5)

fig.show()

In [280]:
dropoff_agg[(dropoff_agg.bookings>=dropoff_agg.bookings.median())
           & (dropoff_agg.unallocation_rate<dropoff_agg.weighted_unallocation_rate.sum()
             )].sort_values('dropoff')[['dropoff']]

,dropoff
3,Ang Mo Kio/Bishan
11,Bedok
12,Bukit Batok
1,Loyang Road
13,Pasir Ris
10,Punggol/Sengkang
8,Upper East Coast Road
2,Woodlands
7,Yishun


In [ ]:
# def quadrantMapping(df):
#     df[(pickup_agg.bookings>=pickup_agg.bookings.median()

In [295]:
pickup_agg.head()

,pickup,distance,bookings,bids,allocated,unallocated,allocation_rate,unallocation_rate,weighted_unallocation_rate
0,City,5.5255,8780,13803,6476,2304,0.737585,0.262415,0.045398
1,Punggol/Sengkang,14.2195,4018,8255,3344,674,0.832255,0.167745,0.013281
2,Woodlands,18.9260,3667,7029,2948,719,0.803927,0.196073,0.014167
3,Ang Mo Kio/Bishan,12.4260,2955,4061,2145,810,0.725888,0.274112,0.015960
4,Jurong West,11.4000,2245,4060,1781,464,0.793318,0.206682,0.009143


In [279]:
pickup_agg[(pickup_agg.bookings>=pickup_agg.bookings.median())
           & (pickup_agg.unallocation_rate<pickup_agg.weighted_unallocation_rate.sum()
             )].sort_values('pickup')[['pickup']]

,pickup
8,Bedok
7,Chua Chu Kang
0,City
6,Fort Road
9,Hougang
4,Jurong West
1,Punggol/Sengkang
14,Tiong Bahru
2,Woodlands
5,Yishun


In [248]:
import plotly.express as px
fig = px.scatter(dropoff_agg, y='unallocation_rate'
                 , x='bookings'
                 , hover_data=['dropoff']
#                  , text="pickup"
                 , log_x=True#, size_max=60
)
fig.update_traces(textposition='top center')

fig.add_vrect(x0=dropoff_agg.bookings.median()
              , x1=dropoff_agg.bookings.median()
              , annotation_text="Median Bookings"
              , annotation_position="top left"
              , line_dash="dot",)


fig.add_hrect(y0=dropoff_agg.weighted_unallocation_rate.sum()
              , y1=dropoff_agg.weighted_unallocation_rate.sum()
              , annotation_text="Unallocation Rate"
              , annotation_position="top right"
              , line_dash="dot",)
fig.update_layout(title='Unallocation Rate by Dropoff Area', title_x = 0.5)
fig.show()

In [281]:
import plotly.express as px
fig = px.scatter(pudo_agg, y='unallocation_rate'
                 , x='bookings'
                 , hover_data=['pudo']
#                  , text="pickup"
#                  , log_x=True#, size_max=60
)
fig.update_traces(textposition='top center')

fig.add_vrect(x0=pudo_agg.bookings.median()
              , x1=pudo_agg.bookings.median()
              , line_dash="dot",)


fig.add_hrect(y0=pudo_agg.weighted_unallocation_rate.sum()
              , y1=pudo_agg.weighted_unallocation_rate.sum()
              , line_dash="dot",)

fig.update_layout(title='Unallocation Rate by PUDO Combination', title_x = 0.5)

fig.show()

In [219]:
import plotly.express as px
fig = px.scatter(pickup_agg, y='unallocation_rate'
                 , x='bookings'
                 , hover_data=['pickup']
                 , text="pickup"
                 , size_max=60)
fig.update_traces(textposition='top center')

fig.add_vrect(x0=pickup_agg.bookings.median()
              , x1=pickup_agg.bookings.median()
#               , annotation_text="Median Bookings"
#               , annotation_position="top left"
              , line_dash="dot",)


fig.add_hrect(y0=pickup_agg.weighted_unallocation_rate.sum()
              , y1=pickup_agg.weighted_unallocation_rate.sum()
#               , annotation_text="Unallocation Rate"
#               , annotation_position="top right"
              , line_dash="dot",)

fig.show()

In [93]:
import plotly.express as px
fig = px.scatter(dropoff_agg, y='allocation_rate', x='bookings', hover_data=['dropoff'])
fig.show()

In [87]:
import plotly.express as px
fig = px.scatter(pudo_agg, y='allocation_rate', x='bookings', hover_data=['pudo'])
fig.show()

In [145]:
import plotly.express as px
fig = px.scatter(distance_agg, y='allocation_rate', x='bookings', hover_data=['distance_ceiled'])
fig.show()

In [150]:
df.head()

,distance,pick_up_time_local,bids,pickup_dis,pickup,drop_off_dis,dropoff,hours,pick_up_lat_round,pick_up_long_round,drop_off_lat_round,drop_off_long_round,bookings,unallocated,allocated,distance_ceiled,pudo
0,17.687,9/2/2015 07:01:25,2,MK20,Seletar,MK02,Farrer Road,7,1.39,103.84,1.30,103.82,1,0,1,18,seletar-farrer_road
1,12.747,12/2/2015 07:19:26,0,MK05,Jurong East,MK02,Farrer Road,7,1.32,103.74,1.30,103.81,1,1,0,13,jurong_east-farrer_road
2,20.560,23/2/2015 07:16:59,0,MK21,Punggol/Sengkang,MK02,Farrer Road,7,1.39,103.90,1.30,103.81,1,1,0,21,punggol/sengkang-farrer_road
3,2.967,18/2/2015 07:59:36,9,TS21,City,MK02,Farrer Road,7,1.30,103.84,1.31,103.82,1,0,1,3,city-farrer_road
4,22.701,2/2/2015 07:30:00,1,MK13,Woodlands,MK02,Farrer Road,7,1.44,103.79,1.30,103.81,1,0,1,23,woodlands-farrer_road


In [160]:
temp = df[['distance_ceiled', 'unallocated', 'bookings']].sort_values('distance_ceiled').reset_index(drop=True)
temp = temp[temp.unallocated>0]
temp = temp[(np.abs(stats.zscore(temp.distance_ceiled)) < 3)]
# temp = temp[(np.abs(stats.zscore(temp.distnce_ceiled)) < 3).all(axis=1)]

temp

,distance_ceiled,unallocated,bookings
0,0,1,1
3,0,1,1
5,0,1,1
6,0,1,1
8,0,1,1
...,...,...,...
50604,38,1,1
50619,38,1,1
50620,38,1,1
50628,39,1,1


In [287]:
import plotly.figure_factory as ff
import numpy as np
np.random.seed(1)

x = temp.distance_ceiled.to_numpy()
hist_data = [x]
group_labels = ['distplot'] # name of the dataset

fig = ff.create_distplot(hist_data, group_labels, show_hist=True
                                         , show_rug=False
)
fig_title = 'Distribution of Unallocation by Distance'
fig.update_layout(title =fig_title
                 , title_x = 0.5
                 , showlegend=False)


fig.show()

In [291]:
distance_agg

,distance_ceiled,bookings,bids,allocated,unallocated,allocation_rate,unallocation_rate,weighted_unallocation_rate
0,0,47,27,18,29,0.382979,0.617021,0.000571
1,1,140,202,90,50,0.642857,0.357143,0.000985
2,2,1126,2234,936,190,0.831261,0.168739,0.003744
3,3,2414,4701,2030,384,0.840928,0.159072,0.007566
4,4,3123,5814,2485,638,0.795709,0.204291,0.012571
5,5,2895,4335,2091,804,0.722280,0.277720,0.015842
6,6,2896,4077,2015,881,0.695787,0.304213,0.017359
7,7,2533,3559,1735,798,0.684959,0.315041,0.015724
8,8,2661,3503,1805,856,0.678316,0.321684,0.016867
9,9,2439,3138,1629,810,0.667897,0.332103,0.015960


In [294]:
import plotly.express as px
fig = px.scatter(distance_agg, y='unallocation_rate'
                 , x='bookings'
                 , hover_data=['distance_ceiled']
#                  , text="distance_ceiled"
                 , size_max=60)
fig.update_traces(textposition='top center')

fig.add_vrect(x0=distance_agg.bookings.median()
              , x1=distance_agg.bookings.median()
#               , annotation_text="Median Bookings"
#               , annotation_position="top left"
              , line_dash="dot",)


fig.add_hrect(y0=distance_agg.weighted_unallocation_rate.sum()
              , y1=distance_agg.weighted_unallocation_rate.sum()
#               , annotation_text="Unallocation Rate"
#               , annotation_position="top right"
              , line_dash="dot",)

fig.show()

In [388]:
area_agg.to_csv('grab_case.csv')